In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

: 

In [ ]:
df= sns.load_dataset('iris')
df

In [ ]:
df.isna().sum()

In [ ]:
df.columns

In [ ]:
#looking at the dtypes
df.info()

In [ ]:
#converting into category
df['species']= df['species'].astype('category')
df.info()

In [ ]:
# looking at the distributions
num_columns= df.select_dtypes('number')

sns.pairplot(num_columns)
plt.title('Distribution for numerical columns')

In [ ]:
#looking at the countplot for our target column
fig, ax= plt.subplots(1, 1, figsize= (8,5))
sns.countplot(data= df, x= 'species', width= 0.5)
plt.title('Distribution of Species')
plt.xlabel('Species')
ax.bar_label(ax.containers[0])


In [ ]:
df.describe()

Doesnt look like it needs normalization

In [ ]:
def softmax(z):
    #z is a m*k matrix here
    # as exp is an unstable function i.e- grows exponentially, we subtract max of z from each z row-wise to prevent overflow. Doesnt affect probabilities
    
    stable_z= z- np.max(z, axis= 1, keepdims= True)
    exp_z = np.exp(stable_z)
    softmax_z= exp_z/ np.sum(exp_z, axis=1, keepdims= True)
    return softmax_z 

In [ ]:
def one_hot_encoding(y):
    unique_cats = np.unique(y.flatten())
    cats_to_index= {cat: i for i, cat in enumerate(unique_cats)}
    encoded= np.zeros((y.shape[0], len(unique_cats)))

    for i, cat in enumerate(y.flatten()):
        encoded[i, cats_to_index[cat]] = 1  
    
    return encoded, cats_to_index


In [ ]:
#adding encoded species column to dataframe
y_encoded, cat_map= one_hot_encoding(df['species'].to_numpy())
new_columns= 'species_'+  np.array(list(cat_map.keys())).astype('str')
encoded_cols_df= pd.DataFrame(data= y_encoded, columns= new_columns)
df= pd.concat([df, encoded_cols_df], axis= 1)

In [ ]:
#verifying the distribution still holds true
df[['species_setosa', 'species_versicolor', 'species_virginica']].apply(lambda col: col.value_counts() ,axis=0)

In [ ]:
X= df[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']].to_numpy()
y= df[new_columns].to_numpy()

In [ ]:
# splitting train and tets data
X_train, X_test, y_train, y_test= train_test_split(X, y, random_state= 42)
print(f'Shape of X_train= {X_train.shape}')
print(f'Shape of X_test= {X_test.shape}')

In [ ]:
#adding a column of ones for bias
X_train= np.c_[np.ones(X_train.shape[0]) ,X_train]

In [ ]:
def compute_cost(p_pred, y_train):
    m= y_train.shape[0]
    return (-1/m)* np.sum( np.sum(y_train * np.log(p_pred), axis= 1))